In [1]:
import numpy as np
import lmadatagen as ldg
print(ldg)

<module 'lmadatagen' from '/Users/SophusOlsen/Desktop/lmadatagen/lmadatagen/__init__.py'>


In [2]:
'''
    Fetch the raw video chunks stored .json files 
    from the disc. We can use cached data if they files
    have been previously loaded.
    
    variables 'joint_indeces', 'joint_keys' and 'meta_keys'
    are automatically imported with LMADataGenerator package.
    
'''

ret_from_disc = False
path_to_json = './json/partial/'

json_util = ldg.LMAJSONUtil(ldg.joint_keys, ldg.joint_indeces, ldg.meta_keys)

# Load the video chunks as a list.
segments = json_util.get_video_segments(
    path_to_json,
    not ret_from_disc,
    ret_from_disc
)

[*]  Processed 191 files

In [3]:
'''
    Compute and extract motion features (based on LMA) from the motion
    data acquired through openpose. Various hyperparameters can be setup
    before extraction.
    
    We can setup hyperparameters for each available feature. The features are:
    [0] weight
    [1] time
    [2] space
    [3] flow
    [4] shape
    
    The hyperparameter is a LIST containig a 3D-tuple with the following meanings:
    
    TUPLE = (feature, effector, weights)
    LIST = [TUPLE0 ... TUPLE4]
    
    The LIST does not need to contain exactly 5 values, but it cannot exceed this limit.
    An effector is joint that is included in the computation of the feature. 
    The weights are a list of values (any numbers) corresponding to how important the
    correspoding effector is. 
    
'''

# Hyperparameter for feature computation
effectors_weights_ = [
    ('time', ['head', 'leftWrist', 'rightWrist'], [0.1, 0.45, 0.45]),
    ('weight', None, np.ones((len(ldg.joint_keys), ))),
]

gestures_per_batch_ = 7

# Setup the parameters (static function call)
ldg.LMARunner.set_lma_hyperparams(
    gestures_per_batch=gestures_per_batch_,
    effectors_weights=effectors_weights_
)

In [4]:
# Extract the motion features by passing in the video
# segments we retrieved above
runner = ldg.LMARunner(segments)

[*]  Processing: [##################################################] 100.0%

In [5]:
# Retrieve the features as a pandas "DataFrame"
motion_features = runner.dataframe()
print(motion_features)

         weight       time     space        flow        shape
0     88.688098   4.378872  0.112384  545.300751   481.038961
1      0.361943   2.249049  0.370871  110.990511  1325.490612
2     21.312160   3.560174  0.453155  335.806578   702.722973
3      2.132541   4.258193  0.216591  473.722960   266.640053
4      2.803054   5.450719  0.153876  538.861774   580.925353
...         ...        ...       ...         ...          ...
1332   1.983928   5.527320  0.243542  396.081096   526.294786
1333   7.431495   6.472742  0.091779  505.621665  4177.964824
1334   4.981218   7.163544  0.215162  523.919876   933.744652
1335  56.452119  12.823697  0.545561  956.074068   244.133830
1336  91.710755  12.039334  0.162681  977.849381   544.525544

[1337 rows x 5 columns]
